In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 15

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')
    if i > 0:
        feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
        monthly_agg[f'rolling_sum_sales_{i + 1}'] = monthly_agg[feats_to_mean].mean(skipna=False, axis=1)
        subset.append(f'rolling_sum_sales_{i + 1}')
        monthly_agg[f'delta_sum_sales_{i}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'prev_sum_sales_{i}']
        subset.append(f'delta_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

In [9]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [10]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,...,prev_sum_sales_13,rolling_sum_sales_14,delta_sum_sales_13,prev_sum_sales_14,rolling_sum_sales_15,delta_sum_sales_14,prev_sum_sales_15,rolling_sum_sales_16,delta_sum_sales_15,target
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,0.0,0.0,0.0,0.0,5.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.357143,0.0,0.0,0.333333,0.0,0.0,0.3125,0.0,0.0


In [11]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold]
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [12]:
feature_cols = subset
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = train_data[train_data['target'] != 0]['target']

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = valid_data[valid_data['target'] != 0]['target']

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [13]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,rolling_sum_sales_3,delta_sum_sales_2,prev_sum_sales_3,rolling_sum_sales_4,delta_sum_sales_3,...,rolling_sum_sales_14,delta_sum_sales_13,prev_sum_sales_14,rolling_sum_sales_15,delta_sum_sales_14,prev_sum_sales_15,rolling_sum_sales_16,delta_sum_sales_15,Company_ID,Product_ID
168600,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2856
2148720,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7101
2148840,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7103
2148900,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7104
168540,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105235,17.0,3.0,10.0,14.0,10.0,10.000000,7.0,4.0,8.50,13.0,...,10.428571,15.0,0.0,9.733333,17.0,0.0,9.1250,17.0,0,1785
838735,13.0,35.0,24.0,-22.0,15.0,21.000000,-2.0,4.0,16.75,9.0,...,11.714286,-20.0,2.0,11.066667,11.0,0.0,10.3750,13.0,0,14226
1627315,0.0,1.0,0.5,-1.0,2.0,1.000000,-2.0,0.0,0.75,0.0,...,1.571429,-2.0,12.0,2.266667,-12.0,0.0,2.1250,0.0,1,12927
909355,100.0,65.0,82.5,35.0,2815.0,993.333333,-2715.0,915.0,973.75,-815.0,...,680.714286,-135.0,5650.0,1012.000000,-5550.0,1290.0,1029.3750,-1190.0,1,751


In [14]:
model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.015,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100
)

0:	learn: 66.8946700	test: 71.3367439	best: 71.3367439 (0)	total: 574ms	remaining: 14m 20s
100:	learn: 29.7476984	test: 32.8994636	best: 32.8994636 (100)	total: 22.9s	remaining: 5m 17s
200:	learn: 24.0244362	test: 27.0648690	best: 27.0648690 (200)	total: 46.3s	remaining: 4m 59s
300:	learn: 22.3687791	test: 25.7599987	best: 25.7599987 (300)	total: 1m 9s	remaining: 4m 38s
400:	learn: 21.2943279	test: 25.0984296	best: 25.0984296 (400)	total: 1m 32s	remaining: 4m 13s
500:	learn: 20.7880266	test: 24.9766394	best: 24.9759492 (499)	total: 1m 56s	remaining: 3m 52s
600:	learn: 20.3692423	test: 24.9166833	best: 24.9160635 (596)	total: 2m 19s	remaining: 3m 28s
700:	learn: 19.9745682	test: 24.8538059	best: 24.8538059 (700)	total: 2m 42s	remaining: 3m 5s
800:	learn: 19.6065944	test: 24.7953300	best: 24.7947825 (799)	total: 3m 6s	remaining: 2m 42s
900:	learn: 19.2904872	test: 24.7559598	best: 24.7553137 (899)	total: 3m 30s	remaining: 2m 19s
1000:	learn: 19.0040490	test: 24.7089880	best: 24.7075337 (

In [15]:
from sklearn.metrics import mean_absolute_error
y_pred_valid = model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_pred_valid)
print("MAE на валидации:", mae_valid)

MAE на валидации: 24.60996395270889


In [16]:
full_train = monthly_agg.dropna(subset=['target'])
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = full_train[full_train['target'] != 0]['target']

final_model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.015,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 67.3386187	total: 187ms	remaining: 4m 39s
100:	learn: 29.9287191	total: 22.4s	remaining: 5m 10s
200:	learn: 24.1638669	total: 44.1s	remaining: 4m 45s
300:	learn: 22.5657440	total: 1m 8s	remaining: 4m 31s
400:	learn: 21.4120677	total: 1m 32s	remaining: 4m 13s
500:	learn: 20.8376962	total: 1m 57s	remaining: 3m 54s
600:	learn: 20.3946620	total: 2m 22s	remaining: 3m 33s
700:	learn: 19.9695448	total: 2m 46s	remaining: 3m 9s
800:	learn: 19.6250221	total: 3m 8s	remaining: 2m 44s
900:	learn: 19.2793506	total: 3m 31s	remaining: 2m 20s
1000:	learn: 18.9631520	total: 3m 54s	remaining: 1m 56s
1100:	learn: 18.6943304	total: 4m 18s	remaining: 1m 33s
1200:	learn: 18.4336524	total: 4m 42s	remaining: 1m 10s
1300:	learn: 18.1973818	total: 5m 4s	remaining: 46.5s
1400:	learn: 17.9874412	total: 5m 26s	remaining: 23.1s
1499:	learn: 17.7746639	total: 5m 48s	remaining: 0us


In [17]:
def predict_next_month(df):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID','Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
        if i > 0:
            feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
            df[f'rolling_sum_sales_{i + 1}'] = df[feats_to_mean].mean(skipna=False, axis=1)
            df[f'delta_sum_sales_{i}'] = df['prev_sum_sales_0'] - df[f'prev_sum_sales_{i}']

    df['Target'] = gb['sum_sales'].shift(-1)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = model.predict(df.loc[to_predict, feature_cols + cat_cols])
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [18]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

,Id,Target,Company_ID,Product_ID,Month_pred
0,0_1_2024-01,0,0,1,2024-01
1,0_2_2024-01,0,0,2,2024-01
2,0_3_2024-01,0,0,3,2024-01
3,0_5_2024-01,0,0,5,2024-01
4,0_6_2024-01,0,0,6,2024-01
...,...,...,...,...,...
113851,3_14664_2024-03,0,3,14664,2024-03
113852,3_14665_2024-03,0,3,14665,2024-03
113853,3_14666_2024-03,0,3,14666,2024-03
113854,3_14667_2024-03,0,3,14667,2024-03


In [19]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag)
    preds.append(p.copy())
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [20]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

100%|██████████| 57652/57652 [00:33<00:00, 1732.74it/s]


In [21]:
sample_submission[['Id', 'Target']]

,Id,Target
0,0_1_2024-01,7.486526
1,0_2_2024-01,0.000000
2,0_3_2024-01,0.000000
3,0_5_2024-01,2.473902
4,0_6_2024-01,3.916521
...,...,...
113851,3_14664_2024-03,0.000000
113852,3_14665_2024-03,190.353185
113853,3_14666_2024-03,0.000000
113854,3_14667_2024-03,0.721185


In [22]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/bruh.csv', index=False)